In [1]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
#import seaborn as sns
import numpy as np
import pandas as pd


SP500=pd.read_csv('SP500_new.csv')
Nasdaq=pd.read_csv('Nasdaq_new.csv')
DJI=pd.read_csv('DJI_new.csv')
DAX=pd.read_csv('DAX_new.csv')
Paris=pd.read_csv('Paris_new.csv')
Tokyo=pd.read_csv('Tokyo_new.csv')
HongKong=pd.read_csv('HongKong_new.csv')
Aus=pd.read_csv('Aus_new.csv')

Data Preprocess

In [2]:
Target=SP500['Adj Close'].pct_change(1).shift(-1)

In [3]:
datasets=[SP500,Nasdaq,DJI,DAX,Paris,Tokyo,HongKong,Aus]
namesets=['SP500','Nasdaq','DJI','DAX','Paris','Tokyo','HongKong','Aus']

In [4]:
def addFeatures(datasets,DaysRet,DaysRetMovAvg,DaysLags):
    """
    compute dataset's DaysReturns,DaysReturnMovAvg and DaysLags
    DaysReturnMovAvg must >=2
    """
    Max=max(DaysRet,DaysRetMovAvg,DaysLags+1)
    for i in range(len(datasets)):
        dataset=datasets[i]
        nameset=namesets[i]
        for i in range(1,DaysRet+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRet'] = dataset['Adj Close'].pct_change(i)
        for i in range(2,DaysRetMovAvg+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRetMovAvg'] = pd.rolling_mean(dataset[nameset+'_'+str(1)+'_'+'DaysRet'], i)
        for i in range(1,DaysLags+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysLags'] = dataset[nameset+'_'+str(1)+'_'+'DaysRet'].shift(i)
        dataset=dataset[Max:]
    return Max

In [5]:
Max=addFeatures(datasets,9,9,9)

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=2).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=3).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=4).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).mean()
C:\Program Files\Anacond

In [6]:
def mergedataset(datasets):
    useless=['Date','Open','High','Low','Close','Adj Close','Volume']
    for i in range(len(datasets)):
        datasets[i]=datasets[i].drop(useless, axis = 1)
    new_dataset = pd.concat(datasets,axis=1)
    return new_dataset

In [7]:
features=mergedataset(datasets)

In [8]:
features=features[Max:-1]
Target=Target[Max:-1]

transfer Target data to binary

In [9]:
Target[Target > 0] = 1
Target[Target <= 0] = 0

Shuffle and Split Data

In [10]:
X_train=features[:-500]
X_test=features[-500:]
y_train=Target[:-500]
y_test=Target[-500:]


#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(features,Target, test_size = 0.2, random_state = 0)

# Modeling

try to predict the the trend of daily return(Classification Problem)

In [24]:
from sklearn.metrics import accuracy_score
def GridSearch_TimeSeries(X_train, y_train,X_test,y_test, algorithm,params_fix, params):
    """
    """
    
    a=params_fix
    total={}
    for i in range(len(list(params.values())[0])):
        for j in range(len(list(params.values())[1])):
            a[list(params.keys())[0]]=list(params.values())[0][i]
            a[list(params.keys())[1]]=list(params.values())[1][j]
        

            reg=algorithm(**a)
            reg.fit(X_train, y_train)
            y_predict=reg.predict(X_test)
            accuracy=accuracy_score(y_test,y_predict)
            #print(accuracies.mean())
            info=[]
            info.append(list(params.values())[0][i])
            info.append(list(params.values())[1][j])
            total[str(accuracy)]=info
    compare=[]
    for i in total.keys():
        compare.append(float(i))
    Best_accu=max(compare)
    print('Best accuracy:',Best_accu)
    print(list(params.keys())[0],':',total[str(Best_accu)][0])
    print(list(params.keys())[1],':',total[str(Best_accu)][1])
    fin_params=params_fix
    fin_params[list(params.keys())[0]]=total[str(Best_accu)][0]
    fin_params[list(params.keys())[1]]=total[str(Best_accu)][1]
    return fin_params

1.RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, RandomForestClassifier,{'random_state':1},{'n_estimators':[120,150],'min_samples_split':[2]})                                                                                                                                              

Best accuracy: 0.512
min_samples_split : 2
n_estimators : 120


In [26]:
print(para)

{'random_state': 1, 'min_samples_split': 2, 'n_estimators': 120}


In [27]:
model=RandomForestClassifier(**para)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=120, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [28]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.512


In [29]:
from sklearn.ensemble import RandomForestClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, RandomForestClassifier,{'random_state':1},{'n_estimators':[80,100,120],'min_samples_split':[2]}) 

model=RandomForestClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.536
min_samples_split : 2
n_estimators : 80
0.536


In [30]:
from sklearn.ensemble import RandomForestClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, RandomForestClassifier,{'random_state':1},{'n_estimators':[30,50,70,80],'min_samples_split':[2]}) 

model=RandomForestClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.536
min_samples_split : 2
n_estimators : 80
0.536


In [32]:
from sklearn.ensemble import RandomForestClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, RandomForestClassifier,{'random_state':1},{'n_estimators':[80],'min_samples_split':[2,5,7,9,11]}) 

model=RandomForestClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.536
min_samples_split : 2
n_estimators : 80
0.536


In [33]:
print(para)

{'random_state': 1, 'min_samples_split': 2, 'n_estimators': 80}


2 KNeighborsClassifier

In [36]:
from sklearn.neighbors import KNeighborsClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, KNeighborsClassifier,{},{'n_neighbors':[23,24,25,26,27,28],'leaf_size':[30]})

Best accuracy: 0.532
n_neighbors : 25
leaf_size : 30


In [37]:
model=KNeighborsClassifier(n_neighbors=25)
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
           weights='uniform')

In [38]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.532


3 SVC

In [40]:
from sklearn.svm import SVC
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test,  SVC,{'random_state':1},{'C':[200,300,400,500,600,700,800,900],'degree':[200,300,400,500,600,700,800,900]})


KeyboardInterrupt: 

In [43]:
from sklearn.svm import SVC
model=SVC(C=400,degree=900)
model.fit(X_train, y_train)

SVC(C=400, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=900, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.512


4 AdaBoostClassifier

In [45]:
from sklearn.ensemble import  AdaBoostClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test,  AdaBoostClassifier,{'random_state':1},{'n_estimators':[72,73,74,75,76,77,78],'learning_rate':[2,2.5,3,3.5,4,4.5,5]})

Best accuracy: 0.528
learning_rate : 3.5
n_estimators : 78


In [46]:
model=AdaBoostClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.528


In [47]:
from sklearn.ensemble import  AdaBoostClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test,  AdaBoostClassifier,{'random_state':1},{'n_estimators':[78,80,85,90,95,100],'learning_rate':[2,2.5,3,3.5,4,4.5,5]})

Best accuracy: 0.528
learning_rate : 3.5
n_estimators : 100


In [48]:
from sklearn.ensemble import  AdaBoostClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test,  AdaBoostClassifier,{'random_state':1},{'n_estimators':[100,150,200,250,300,400,500],'learning_rate':[3.5]})

Best accuracy: 0.528
learning_rate : 3.5
n_estimators : 500


In [49]:
print(para)

{'random_state': 1, 'learning_rate': 3.5, 'n_estimators': 500}


5 GradientBoostingClassifier

5.1 benchmark

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [52]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.51


5.2 Fix learning rate and number of estimators for tuning tree-based parameters

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'min_samples_split':100,'min_samples_leaf':50,'max_depth':8,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'n_estimators':[20,30,40,50,60,70,80],'learning_rate':[0.1]})

Best accuracy: 0.522
learning_rate : 0.1
n_estimators : 50


In [54]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.522


In [55]:
print(para)

{'max_depth': 8, 'random_state': 10, 'min_samples_split': 100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 50, 'subsample': 0.8, 'min_samples_leaf': 50}


5.3 Tune max_depth and num_samples_split

In [56]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'max_depth': 8, 'random_state': 10, 'min_samples_split': 100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 50, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'max_depth':[5,7,9,11,13,15],'min_samples_split':[200,400,600,800,1000]})

Best accuracy: 0.534
max_depth : 9
min_samples_split : 1000


In [57]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[8,9,10],'min_samples_split':[1000,1250,1500,1750,2000]})

Best accuracy: 0.542
max_depth : 10
min_samples_split : 1000


In [58]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[9,10,11],'min_samples_split':[900,950,1000,1050,1100]})

Best accuracy: 0.548
max_depth : 9
min_samples_split : 1100


In [59]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[7,8,9,10,11],'min_samples_split':[900,950,1000,1050,1100,1200,1300,1400,1500]})

Best accuracy: 0.548
max_depth : 9
min_samples_split : 1100


In [62]:
print(para)

{'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50}


In [61]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.548


5.4 Tune min_samples_leaf

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'min_samples_split':[1100],'min_samples_leaf':[30,40,50,60,70]})

Best accuracy: 0.548
min_samples_split : 1100
min_samples_leaf : 50


In [65]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'min_samples_split':[1100],'min_samples_leaf':[45,47,50,53,55]})

Best accuracy: 0.548
min_samples_split : 1100
min_samples_leaf : 50


In [66]:
print(para)

{'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50}


In [67]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.548


5.5 Tune max_features

In [68]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'min_samples_leaf':[50],'max_features':[7,9,11,13,15,17,19,21]})

Best accuracy: 0.544
max_features : 9
min_samples_leaf : 50


In [69]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'min_samples_leaf':[50],'max_features':[17,18,19,20,21,22,23]})

Best accuracy: 0.54
max_features : 17
min_samples_leaf : 50


In [72]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'min_samples_leaf':[50],'max_features':['sqrt']})

Best accuracy: 0.548
max_features : sqrt
min_samples_leaf : 50


In [73]:
print(para)

{'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50}


In [74]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.548


5.6 Tuning subsample

In [77]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'subsample':[0.80,0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89],'min_samples_leaf':[50]})




Best accuracy: 0.548
subsample : 0.8
min_samples_leaf : 50


In [80]:
from sklearn.ensemble import GradientBoostingClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50},
                    {'subsample':[0.78,0.79,0.80],'min_samples_leaf':[50]})




Best accuracy: 0.548
subsample : 0.8
min_samples_leaf : 50


In [81]:
print(para)

{'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 30, 'subsample': 0.8, 'min_samples_leaf': 50}


In [82]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.548


In [ ]:
5.7 lower learning rate

In [108]:
params={'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 29, 'subsample': 0.8, 'min_samples_leaf': 50}


In [109]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.55


In [110]:
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'max_features': 'sqrt','subsample': 0.8, 'min_samples_leaf': 50},
                    {'learning_rate':[0.08,0.09,0.1,0.11,0.12],'n_estimators':[25,26,27,28,29,30,31,32,33]})

Best accuracy: 0.55
learning_rate : 0.1
n_estimators : 33


In [113]:
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, 
                    GradientBoostingClassifier,
                    {'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'max_features': 'sqrt','subsample': 0.8, 'min_samples_leaf': 50},
                    {'learning_rate':[0.1],'n_estimators':[33,32,31,34,35]})

Best accuracy: 0.55
learning_rate : 0.1
n_estimators : 33


In [114]:
print(para)

{'random_state': 10, 'max_depth': 9, 'min_samples_split': 1100, 'learning_rate': 0.1, 'max_features': 'sqrt', 'n_estimators': 33, 'subsample': 0.8, 'min_samples_leaf': 50}


In [117]:
GradientBoostingClassifier(**para)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [115]:
model=GradientBoostingClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.55


6 QuadraticDiscriminantAnalysis

In [131]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model=QuadraticDiscriminantAnalysis()
model.fit(X_train, y_train)

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariances=False, tol=0.0001)

In [133]:


y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.498


7 SGDClassifier

In [123]:
from sklearn.linear_model import SGDClassifier
para=GridSearch_TimeSeries(X_train, y_train,X_test,y_test, SGDClassifier,{'random_state':1},{'n_iter':[3,5,7,10,13,15],'l1_ratio':[0.13,0.14,0.15,0.16,0.17,0.18]})

Best accuracy: 0.524
n_iter : 13
l1_ratio : 0.18


In [129]:
model=SGDClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.524


In [130]:
SGDClassifier(**para)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=1.0, learning_rate='optimal',
       loss='hinge', n_iter=13, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=1, shuffle=True, verbose=0, warm_start=False)

In [ ]:
1.GradientBoostingClassifier(criterion='friedman_mse', init=None,                          0.55
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)
2.RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)           0.536
3.KNeighborsClassifier(n_neighbors=25)                                                     0.532
4.AdaBoostClassifier(random_state=1,n_estimators=100,learning_rate=3.5)                    0.528
5.SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,               0.524
       eta0=0.0, fit_intercept=True, l1_ratio=1.0, learning_rate='optimal',
       loss='hinge', n_iter=13, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=1, shuffle=True, verbose=0, warm_start=False)
6.SVC(random_state=1,C=400,degree=900)                                                     0.512

# Stacking models

In [134]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score


In [135]:
'''Train the stacked models then predict the test data'''
#ORIGINAL
class StackingEstimator(BaseEstimator, TransformerMixin):
        
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

Trial_1

In [137]:

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=25)),
    StackingEstimator(estimator=RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)),
    GradientBoostingClassifier(criterion='friedman_mse', init=None,                         
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

)

In [138]:
stacked_pipeline.fit(X_train, y_train)
y_predict=stacked_pipeline.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.478


Trial_2

In [139]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=AdaBoostClassifier(random_state=1,n_estimators=100,learning_rate=3.5)),
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=25)),
    StackingEstimator(estimator=RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)),
    GradientBoostingClassifier(criterion='friedman_mse', init=None,                         
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

)

stacked_pipeline.fit(X_train, y_train)
y_predict=stacked_pipeline.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.538


Trial_3

In [140]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=1.0, learning_rate='optimal',
       loss='hinge', n_iter=13, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=1, shuffle=True, verbose=0, warm_start=False)),
    StackingEstimator(estimator=AdaBoostClassifier(random_state=1,n_estimators=100,learning_rate=3.5)),
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=25)),
    StackingEstimator(estimator=RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)),
    GradientBoostingClassifier(criterion='friedman_mse', init=None,                         
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

)

stacked_pipeline.fit(X_train, y_train)
y_predict=stacked_pipeline.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.468


In [ ]:
Trial_4

In [141]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=SVC(random_state=1,C=400,degree=900)),
    StackingEstimator(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=1.0, learning_rate='optimal',
       loss='hinge', n_iter=13, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=1, shuffle=True, verbose=0, warm_start=False)),
    StackingEstimator(estimator=AdaBoostClassifier(random_state=1,n_estimators=100,learning_rate=3.5)),
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=25)),
    StackingEstimator(estimator=RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)),
    GradientBoostingClassifier(criterion='friedman_mse', init=None,                         
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

)

stacked_pipeline.fit(X_train, y_train)
y_predict=stacked_pipeline.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.498


# Tune Stacking model

In [ ]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=AdaBoostClassifier(random_state=1,n_estimators=100,learning_rate=3.5)),
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=25)),
    StackingEstimator(estimator=RandomForestClassifier(random_state=1, min_samples_split= 2, n_estimators= 80)),
    GradientBoostingClassifier(criterion='friedman_mse', init=None,                         
              learning_rate=0.1, loss='deviance', max_depth=9,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=1100, min_weight_fraction_leaf=0.0,
              n_estimators=33, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

)